In [9]:
import numpy as np
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #crop
    transforms.RandomHorizontalFlip(),   #flip
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


trainset = torchvision.datasets.CIFAR100(root='./data1', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data1', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [11]:
#res31 = F.interpolate(x, (8, 8), scale_factor=None, mode='nearest', align_corners=None)

def downsample(in_channels, out_channels,kernel_size, stride,padding):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, 
                     stride, padding, bias=False), nn.BatchNorm2d(out_channels))


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.relu = nn.ReLU(inplace = True)
        
        self.conv1 =   nn.Conv2d(3, 32,    3,1,1)
        self.conv2 =   nn.Conv2d(32, 32,   3,1,1)
        self.conv3_1 = nn.Conv2d(32, 64,   3,2,1)
        self.conv3 =   nn.Conv2d(64, 64,   3,1,1)
        self.conv4_1 = nn.Conv2d(64, 128,  3,2,1)
        self.conv4 =   nn.Conv2d(128, 128, 3,1,1)
        self.conv5_1 = nn.Conv2d(128, 256, 3,2,1)
        self.conv5 =   nn.Conv2d(256, 256, 3,1,1)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.drop = nn.Dropout(p = 0.6)
        
        self.pool = nn.MaxPool2d(4,4)
        
        #试一下（x,x,1,2,0）no good!
        
        self.downsample1 = downsample(32,64,2,2,0)
        self.downsample2 = downsample(64,128,2,2,0)
        self.downsample3 = downsample(128,256,2,2,0)
        
        self.fc = nn.Linear(256*1*1, 100)

    def forward(self, x):
        #conv1
        x = self.conv1(x)               #self.conv1 =   nn.Conv2d(3, 32,    3,1,1)
        x = self.relu(self.bn1(x))
        x = self.drop(x)
        
        #B1 (no downsample)
        res11 = x
        x = self.conv2(x)            #self.conv2 =   nn.Conv2d(32, 32,   3,1,1)
        x = self.relu(self.bn2(x))
        x = self.conv2(x)
        x = self.bn2(x)
        x = res11 + x
        
        res12 = x
        x = self.conv2(x)
        x = self.relu(self.bn2(x))
        x = self.conv2(x)
        x = self.bn2(x)
        x = res12 + x
        
        #B2
        res21 = x
        x = self.conv3_1(x)             #self.conv3_1 = nn.Conv2d(32, 64,   3,2,1)
        x = self.relu(self.bn3(x))
        x = self.conv3(x)               #self.conv3 =   nn.Conv2d(64, 64,   3,1,1)
        x = self.bn3(x)
        res21 = self.downsample1(res21)
        x = res21 + x
        
        res22 = x
        x = self.conv3(x)
        x = self.relu(self.bn3(x))
        x = self.conv3(x)
        x = self.bn3(x)
        x = res22 + x
        
        res23 = x
        x = self.conv3(x)
        x = self.relu(self.bn3(x))
        x = self.conv3(x)
        x = self.bn3(x)
        x = res23 + x
        
        res24 = x
        x = self.conv3(x)
        x = self.relu(self.bn3(x))
        x = self.conv3(x)
        x = self.bn3(x)
        x = res24 + x
        
        
        #B3  
        res31 = x
        x = self.conv4_1(x)     #self.conv4_1 = nn.Conv2d(64, 128,  3,2,1)
        x = self.relu(self.bn4(x))
        x = self.conv4(x)          #self.conv4 =   nn.Conv2d(128, 128, 3,1,1)
        x = self.bn4(x)
        res31 = self.downsample2(res31)
        x = res31 + x
        
        res32 = x
        x = self.conv4(x)
        x = self.relu(self.bn4(x))
        x = self.conv4(x)
        x = self.bn4(x)
        x = res32 + x
        
        res33 = x
        x = self.conv4(x)
        x = self.relu(self.bn4(x))
        x = self.conv4(x)
        x = self.bn4(x)
        x = res33 + x
        
        res34 = x
        x = self.conv4(x)
        x = self.relu(self.bn4(x))
        x = self.conv4(x)
        x = self.bn4(x)
        x = res34 + x
        
        #B4
        res41 = x
        x = self.conv5_1(x)                    #self.conv5_1 = nn.Conv2d(128, 256, 3,2,1)
        x = self.relu(self.bn5(x))
        x = self.conv5(x)                      #self.conv5 =   nn.Conv2d(256, 256, 3,1,1)
        x = self.bn5(x)
        res41 = self.downsample3(res41)
        x = res41 + x
        
        res42 = x
        x = self.relu(self.bn5(x))
        x = self.conv5(x)
        x = self.bn5(x)
        x = res42 + x
        
        
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x
        
        
        

model = Model().to(device)

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# Assuming optimizer has two groups.
#lambda1 = lambda epoch: epoch // 30
#lambda2 = lambda epoch: 0.9 ** epoch
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

                                                                # 0.001  0.0001 0.00001
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[    50,      80,     110 ], gamma=0.1)


#  TRAINING
model.train()
epoch = 130
time1 = time.time()

for epoch in range(epoch):  # loop over the dataset multiple times
    scheduler.step()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        #print statistics
#         running_loss += loss.item()
#         if i % 10 == 9:    # print every 10 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                  (epoch + 1, i + 1, running_loss / 10))
#             running_loss = 0.0# print statistics
        
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch',epoch ,'Accuracy of the network on the 10000 test images: %f %%' % (
        100 * correct / total))

     
print('Finished Training')
time2 = time.time()
print(time2-time1)

Epoch 0 Accuracy of the network on the 10000 test images: 3.690000 %
Epoch 1 Accuracy of the network on the 10000 test images: 21.850000 %
Epoch 2 Accuracy of the network on the 10000 test images: 26.900000 %
Epoch 3 Accuracy of the network on the 10000 test images: 29.760000 %
Epoch 4 Accuracy of the network on the 10000 test images: 35.100000 %
Epoch 5 Accuracy of the network on the 10000 test images: 36.210000 %
Epoch 6 Accuracy of the network on the 10000 test images: 39.480000 %
Epoch 7 Accuracy of the network on the 10000 test images: 42.260000 %
Epoch 8 Accuracy of the network on the 10000 test images: 43.240000 %
Epoch 9 Accuracy of the network on the 10000 test images: 44.620000 %
Epoch 10 Accuracy of the network on the 10000 test images: 46.190000 %
Epoch 11 Accuracy of the network on the 10000 test images: 47.630000 %
Epoch 12 Accuracy of the network on the 10000 test images: 47.840000 %
Epoch 13 Accuracy of the network on the 10000 test images: 49.030000 %
Epoch 14 Accuracy

Epoch 116 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 117 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 118 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 119 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 120 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 121 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 122 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 123 Accuracy of the network on the 10000 test images: 60.130000 %
Epoch 124 Accuracy of the network on the 10000 test images: 60.130000 %
Epoch 125 Accuracy of the network on the 10000 test images: 60.120000 %
Epoch 126 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 127 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 128 Accuracy of the network on the 10000 test images: 60.110000 %
Epoch 129 Accuracy of the network on the 10000 test images: 60.1

On wednesday night, tried with downsample(x,x,2,2,0)
Epoch 57 Accuracy of the network on the 10000 test images: 58.150000 %

On thursday noon, tried with downsample(x,x,4,2,1)
Epoch 122 Accuracy of the network on the 10000 test images: 60.030000 %

On thursday 6pm, tried with downsample(x,x,8,2,3)
Epoch 117 Accuracy of the network on the 10000 test images: 59.340000 %

In [13]:
#correct = 0
#total = 0
#with torch.no_grad():
#    for data in testloader:
#        inputs, labels = inputs.to(device), labels.to(device)
#        outputs = model(inputs)
#        _, predicted = torch.max(outputs.data, 1)
#        total += labels.size(0)
#        correct += (predicted == labels).sum().item()

#print('Accuracy of the network on the 10000 test images: %d %%' % (
#    100 * correct / total))

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 60.13 %


Epoch 0 Accuracy of the network on the 10000 test images: 3.690000 %
Epoch 1 Accuracy of the network on the 10000 test images: 21.850000 %
Epoch 2 Accuracy of the network on the 10000 test images: 26.900000 %
Epoch 3 Accuracy of the network on the 10000 test images: 29.760000 %
Epoch 4 Accuracy of the network on the 10000 test images: 35.100000 %
Epoch 5 Accuracy of the network on the 10000 test images: 36.210000 %
Epoch 6 Accuracy of the network on the 10000 test images: 39.480000 %
Epoch 7 Accuracy of the network on the 10000 test images: 42.260000 %
Epoch 8 Accuracy of the network on the 10000 test images: 43.240000 %
Epoch 9 Accuracy of the network on the 10000 test images: 44.620000 %
Epoch 10 Accuracy of the network on the 10000 test images: 46.190000 %
Epoch 11 Accuracy of the network on the 10000 test images: 47.630000 %
Epoch 12 Accuracy of the network on the 10000 test images: 47.840000 %
Epoch 13 Accuracy of the network on the 10000 test images: 49.030000 %
Epoch 14 Accuracy of the network on the 10000 test images: 50.170000 %
Epoch 15 Accuracy of the network on the 10000 test images: 50.500000 %
Epoch 16 Accuracy of the network on the 10000 test images: 50.610000 %
Epoch 17 Accuracy of the network on the 10000 test images: 52.200000 %
Epoch 18 Accuracy of the network on the 10000 test images: 52.300000 %
Epoch 19 Accuracy of the network on the 10000 test images: 52.230000 %
Epoch 20 Accuracy of the network on the 10000 test images: 53.840000 %
Epoch 21 Accuracy of the network on the 10000 test images: 54.190000 %
Epoch 22 Accuracy of the network on the 10000 test images: 54.410000 %
Epoch 23 Accuracy of the network on the 10000 test images: 54.530000 %
Epoch 24 Accuracy of the network on the 10000 test images: 54.700000 %
Epoch 25 Accuracy of the network on the 10000 test images: 55.000000 %
Epoch 26 Accuracy of the network on the 10000 test images: 55.390000 %
Epoch 27 Accuracy of the network on the 10000 test images: 55.490000 %
Epoch 28 Accuracy of the network on the 10000 test images: 55.120000 %
Epoch 29 Accuracy of the network on the 10000 test images: 54.920000 %
Epoch 30 Accuracy of the network on the 10000 test images: 55.700000 %
Epoch 31 Accuracy of the network on the 10000 test images: 56.240000 %
Epoch 32 Accuracy of the network on the 10000 test images: 55.310000 %
Epoch 33 Accuracy of the network on the 10000 test images: 56.330000 %
Epoch 34 Accuracy of the network on the 10000 test images: 57.050000 %
Epoch 35 Accuracy of the network on the 10000 test images: 56.710000 %
Epoch 36 Accuracy of the network on the 10000 test images: 55.980000 %
Epoch 37 Accuracy of the network on the 10000 test images: 56.360000 %
Epoch 38 Accuracy of the network on the 10000 test images: 57.050000 %
Epoch 39 Accuracy of the network on the 10000 test images: 57.620000 %
Epoch 40 Accuracy of the network on the 10000 test images: 56.850000 %
Epoch 41 Accuracy of the network on the 10000 test images: 56.910000 %
Epoch 42 Accuracy of the network on the 10000 test images: 57.390000 %
Epoch 43 Accuracy of the network on the 10000 test images: 57.450000 %
Epoch 44 Accuracy of the network on the 10000 test images: 57.980000 %
Epoch 45 Accuracy of the network on the 10000 test images: 57.780000 %
Epoch 46 Accuracy of the network on the 10000 test images: 56.700000 %
Epoch 47 Accuracy of the network on the 10000 test images: 56.940000 %
Epoch 48 Accuracy of the network on the 10000 test images: 56.530000 %
Epoch 49 Accuracy of the network on the 10000 test images: 57.780000 %
Epoch 50 Accuracy of the network on the 10000 test images: 59.630000 %
Epoch 51 Accuracy of the network on the 10000 test images: 60.080000 %
Epoch 52 Accuracy of the network on the 10000 test images: 59.990000 %
Epoch 53 Accuracy of the network on the 10000 test images: 59.710000 %
Epoch 54 Accuracy of the network on the 10000 test images: 60.110000 %
Epoch 55 Accuracy of the network on the 10000 test images: 59.850000 %
Epoch 56 Accuracy of the network on the 10000 test images: 59.920000 %
Epoch 57 Accuracy of the network on the 10000 test images: 60.440000 %
Epoch 58 Accuracy of the network on the 10000 test images: 60.300000 %
Epoch 59 Accuracy of the network on the 10000 test images: 60.000000 %
Epoch 60 Accuracy of the network on the 10000 test images: 60.230000 %
Epoch 61 Accuracy of the network on the 10000 test images: 60.100000 %
Epoch 62 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 63 Accuracy of the network on the 10000 test images: 60.010000 %
Epoch 64 Accuracy of the network on the 10000 test images: 59.650000 %
Epoch 65 Accuracy of the network on the 10000 test images: 59.850000 %
Epoch 66 Accuracy of the network on the 10000 test images: 59.810000 %
Epoch 67 Accuracy of the network on the 10000 test images: 59.940000 %
Epoch 68 Accuracy of the network on the 10000 test images: 60.070000 %
Epoch 69 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 70 Accuracy of the network on the 10000 test images: 60.050000 %
Epoch 71 Accuracy of the network on the 10000 test images: 60.160000 %
Epoch 72 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 73 Accuracy of the network on the 10000 test images: 59.890000 %
Epoch 74 Accuracy of the network on the 10000 test images: 60.250000 %
Epoch 75 Accuracy of the network on the 10000 test images: 59.860000 %
Epoch 76 Accuracy of the network on the 10000 test images: 59.990000 %
Epoch 77 Accuracy of the network on the 10000 test images: 59.870000 %
Epoch 78 Accuracy of the network on the 10000 test images: 60.050000 %
Epoch 79 Accuracy of the network on the 10000 test images: 59.710000 %
Epoch 80 Accuracy of the network on the 10000 test images: 59.900000 %
Epoch 81 Accuracy of the network on the 10000 test images: 60.020000 %
Epoch 82 Accuracy of the network on the 10000 test images: 59.970000 %
Epoch 83 Accuracy of the network on the 10000 test images: 60.010000 %
Epoch 84 Accuracy of the network on the 10000 test images: 60.060000 %
Epoch 85 Accuracy of the network on the 10000 test images: 59.980000 %
Epoch 86 Accuracy of the network on the 10000 test images: 60.020000 %
Epoch 87 Accuracy of the network on the 10000 test images: 59.960000 %
Epoch 88 Accuracy of the network on the 10000 test images: 60.100000 %
Epoch 89 Accuracy of the network on the 10000 test images: 60.030000 %
Epoch 90 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 91 Accuracy of the network on the 10000 test images: 60.130000 %
Epoch 92 Accuracy of the network on the 10000 test images: 60.110000 %
Epoch 93 Accuracy of the network on the 10000 test images: 60.240000 %
Epoch 94 Accuracy of the network on the 10000 test images: 60.240000 %
Epoch 95 Accuracy of the network on the 10000 test images: 60.350000 %
Epoch 96 Accuracy of the network on the 10000 test images: 60.290000 %
Epoch 97 Accuracy of the network on the 10000 test images: 60.200000 %
Epoch 98 Accuracy of the network on the 10000 test images: 60.240000 %
Epoch 99 Accuracy of the network on the 10000 test images: 60.310000 %
Epoch 100 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 101 Accuracy of the network on the 10000 test images: 60.250000 %
Epoch 102 Accuracy of the network on the 10000 test images: 60.100000 %
Epoch 103 Accuracy of the network on the 10000 test images: 60.160000 %
Epoch 104 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 105 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 106 Accuracy of the network on the 10000 test images: 60.190000 %
Epoch 107 Accuracy of the network on the 10000 test images: 60.210000 %
Epoch 108 Accuracy of the network on the 10000 test images: 60.250000 %
Epoch 109 Accuracy of the network on the 10000 test images: 60.210000 %
Epoch 110 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 111 Accuracy of the network on the 10000 test images: 60.200000 %
Epoch 112 Accuracy of the network on the 10000 test images: 60.200000 %
Epoch 113 Accuracy of the network on the 10000 test images: 60.190000 %
Epoch 114 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 115 Accuracy of the network on the 10000 test images: 60.190000 %
Epoch 116 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 117 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 118 Accuracy of the network on the 10000 test images: 60.170000 %
Epoch 119 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 120 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 121 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 122 Accuracy of the network on the 10000 test images: 60.090000 %
Epoch 123 Accuracy of the network on the 10000 test images: 60.130000 %
Epoch 124 Accuracy of the network on the 10000 test images: 60.130000 %
Epoch 125 Accuracy of the network on the 10000 test images: 60.120000 %
Epoch 126 Accuracy of the network on the 10000 test images: 60.150000 %
Epoch 127 Accuracy of the network on the 10000 test images: 60.180000 %
Epoch 128 Accuracy of the network on the 10000 test images: 60.110000 %
Epoch 129 Accuracy of the network on the 10000 test images: 60.130000 %
Finished Training
3957.5201094150543